In [1]:


import copy
%matplotlib inline

import refnx
from refnx.reflect import SLD
from refnx.analysis import Parameter
from hogben.models.samples import Sample
from hogben.optimise import optimise_parameters
from hogben.models.bilayers import BilayerDPPC
from hogben.utils import Fisher


In [5]:
def simple_sample():
    """Define a bilayer sample, and return the associated refnx model"""
    # Define sample

    layer1_thick = Parameter(70, 'Layer1 thickness', (50, 90))
    layer2_thick = Parameter(40, 'Layer2 thickness', (30, 50))
    layer1_rough = Parameter(5, 'Layer1 Roughness', (2, 10))
    layer2_rough = Parameter(6, 'Layer2 roughness', (2, 10))

    ul_thick = Parameter(125, 'Underlayer Thickness', (0, 250))
    ul_sld = Parameter(4, 'Underlayer SLD', (1.5, 7))

    air = refnx.reflect.SLD(0, name='Air')    

    layer1 = SLD(6.5, name="Layer 1")(thick=layer1_thick, rough=layer1_rough)
    layer2 = SLD(1.5, name="Layer 2")(thick=layer2_thick, rough=layer2_rough)
    underlayer = SLD(ul_sld, name="Underlayer SLD")(thick=ul_thick, rough=0)

    substrate = SLD(2.074, name='Substrate')(rough=5)


    params = [
        layer1_thick,
        layer2_thick,
        layer1_rough,
        layer2_rough,
    ]

    for param in params:
        param.vary = True

    optimize = [
       ul_thick,
       ul_sld,
    ]

    for params in optimize:
        params.optimize = True

    sample = air | layer2 | layer1 | layer2 | underlayer | substrate
    return sample
structure = simple_sample()
print(type(structure))
sample = Sample(simple_sample())

<class 'refnx.reflect.structure.Structure'>


TypeError: Can't instantiate abstract class Sample with abstract methods reflectivity_profile, sld_profile

In [ ]:
angle_times = [(0.7, 100, 100),
               (2.3, 100, 400),
               ]    
sample.polarised = False
optimise_parameters(sample, angle_times)